In [28]:
import pandas as pd
import matplotlib.pyplot as plt

# Load the CSV file
file_path = '/scratch/baj321/MIMIC-Note/physionet.org/files/mimic-iv-note/2.2/note/radiology.csv'
df_sample = pd.read_csv(file_path)

# Convert 'charttime' to datetime
df_sample['charttime'] = pd.to_datetime(df_sample['charttime'])

# Display the first few rows of the dataframe to get an overview
print("First few rows of the dataframe (sample):")
display(df_sample.head())

# Print the column types
print("Column types:")
print(df_sample.dtypes)


First few rows of the dataframe (sample):


,note_id,subject_id,hadm_id,note_type,note_seq,charttime,storetime,text
0,10000032-RR-14,10000032,22595853.0,RR,14,2180-05-06 21:19:00,2180-05-06 23:32:00,EXAMINATION: CHEST (PA AND LAT)\n\nINDICATION...
1,10000032-RR-15,10000032,22595853.0,RR,15,2180-05-06 23:00:00,2180-05-06 23:26:00,EXAMINATION: LIVER OR GALLBLADDER US (SINGLE ...
2,10000032-RR-16,10000032,22595853.0,RR,16,2180-05-07 09:55:00,2180-05-07 11:15:00,"INDICATION: ___ HCV cirrhosis c/b ascites, hi..."
3,10000032-RR-18,10000032,NaN,RR,18,2180-06-03 12:46:00,2180-06-03 14:01:00,EXAMINATION: Ultrasound-guided paracentesis.\...
4,10000032-RR-20,10000032,NaN,RR,20,2180-07-08 13:18:00,2180-07-08 14:15:00,EXAMINATION: Paracentesis\n\nINDICATION: ___...


Column types:
note_id               object
subject_id             int64
hadm_id              float64
note_type             object
note_seq               int64
charttime     datetime64[ns]
storetime             object
text                  object
dtype: object


In [32]:
import pandas as pd
import matplotlib.pyplot as plt

# Load a sample of the CSV file first to get an overview
file_path = '/scratch/baj321/MIMIC-Note/physionet.org/files/mimic-iv-note/2.2/note/radiology.csv'
# Display the first few rows of the dataframe to get an overview
df_sample = pd.read_csv(file_path)

print("First few rows of the dataframe (sample):")
display(df_sample.head())

First few rows of the dataframe (sample):


,note_id,subject_id,hadm_id,note_type,note_seq,charttime,storetime,text
0,10000032-RR-14,10000032,22595853.0,RR,14,2180-05-06 21:19:00,2180-05-06 23:32:00,EXAMINATION: CHEST (PA AND LAT)\n\nINDICATION...
1,10000032-RR-15,10000032,22595853.0,RR,15,2180-05-06 23:00:00,2180-05-06 23:26:00,EXAMINATION: LIVER OR GALLBLADDER US (SINGLE ...
2,10000032-RR-16,10000032,22595853.0,RR,16,2180-05-07 09:55:00,2180-05-07 11:15:00,"INDICATION: ___ HCV cirrhosis c/b ascites, hi..."
3,10000032-RR-18,10000032,NaN,RR,18,2180-06-03 12:46:00,2180-06-03 14:01:00,EXAMINATION: Ultrasound-guided paracentesis.\...
4,10000032-RR-20,10000032,NaN,RR,20,2180-07-08 13:18:00,2180-07-08 14:15:00,EXAMINATION: Paracentesis\n\nINDICATION: ___...


In [33]:
print(df_sample.iloc[1]['text'])

EXAMINATION:  LIVER OR GALLBLADDER US (SINGLE ORGAN)

INDICATION:  ___ year-old female with cirrhosis, jaundice.

TECHNIQUE:  Grey scale and color Doppler ultrasound images of the abdomen were
obtained.

COMPARISON:  None.

FINDINGS: 

LIVER: The liver is coarsened and nodular in echotexture. There is no focal
liver mass. Main portal vein and its major branches are patent with normal
hepatopetal flow.  The main hepatic artery shows normal arterial waveform. 
There is a small amount of ascites.

BILE DUCTS: There is no intrahepatic biliary dilation. The CBD measures 4 mm.

GALLBLADDER: The gallbladder is contracted with a shadowing gallstone.

PANCREAS: Imaged portion of the pancreas appears within normal limits, without
masses or pancreatic ductal dilation, with portions of the pancreatic tail
obscured by overlying bowel gas.

SPLEEN: Normal echogenicity, measuring 13.5 cm.

KIDNEYS: The right kidney measures 12.1 cm. The left kidney measures 13.4 cm.
Normal cortical echogenicity and c

In [4]:
!pip install transformers

Defaulting to user installation because normal site-packages is not writeable


In [1]:
from arguments import args_parser

In [2]:
from __future__ import absolute_import
from __future__ import print_function

import numpy as np
import platform
import pickle
import json
import os


class Discretizer:
    def __init__(self, timestep=0.8, store_masks=True, impute_strategy='zero', start_time='zero',
                 config_path= 'ehr_utils/resources/discretizer_config.json'):

        with open(config_path) as f:
            config = json.load(f)
            self._id_to_channel = config['id_to_channel']
            self._channel_to_id = dict(zip(self._id_to_channel, range(len(self._id_to_channel))))
            self._is_categorical_channel = config['is_categorical_channel']
            self._possible_values = config['possible_values']
            self._normal_values = config['normal_values']

        self._header = ["Hours"] + self._id_to_channel
        self._timestep = timestep
        self._store_masks = store_masks
        self._start_time = start_time
        self._impute_strategy = impute_strategy

        # for statistics
        self._done_count = 0
        self._empty_bins_sum = 0
        self._unused_data_sum = 0

    def transform(self, X, header=None, end=None):
        if header is None:
            header = self._header
        assert header[0] == "Hours"
        eps = 1e-6

        N_channels = len(self._id_to_channel)
        ts = [float(row[0]) for row in X]
        for i in range(len(ts) - 1):
            assert ts[i] < ts[i+1] + eps

        if self._start_time == 'relative':
            first_time = ts[0]
        elif self._start_time == 'zero':
            first_time = 0
        else:
            raise ValueError("start_time is invalid")

        if end is None:
            max_hours = max(ts) - first_time
        else:
            max_hours = end - first_time

        N_bins = int(max_hours / self._timestep + 1.0 - eps)

        cur_len = 0
        begin_pos = [0 for i in range(N_channels)]
        end_pos = [0 for i in range(N_channels)]
        for i in range(N_channels):
            channel = self._id_to_channel[i]
            begin_pos[i] = cur_len
            if self._is_categorical_channel[channel]:
                end_pos[i] = begin_pos[i] + len(self._possible_values[channel])
            else:
                end_pos[i] = begin_pos[i] + 1
            cur_len = end_pos[i]

        data = np.zeros(shape=(N_bins, cur_len), dtype=float)
        mask = np.zeros(shape=(N_bins, N_channels), dtype=int)
        original_value = [["" for j in range(N_channels)] for i in range(N_bins)]
        total_data = 0
        unused_data = 0

        def write(data, bin_id, channel, value, begin_pos):
            channel_id = self._channel_to_id[channel]
            if self._is_categorical_channel[channel]:
                category_id = self._possible_values[channel].index(value)
                N_values = len(self._possible_values[channel])
                one_hot = np.zeros((N_values,))
                one_hot[category_id] = 1
                for pos in range(N_values):
                    data[bin_id, begin_pos[channel_id] + pos] = one_hot[pos]
            else:
                data[bin_id, begin_pos[channel_id]] = float(value)

        for row in X:
            t = float(row[0]) - first_time
            if t > max_hours + eps:
                continue
            bin_id = int(t / self._timestep - eps)
            assert 0 <= bin_id < N_bins

            for j in range(1, len(row)):
                if row[j] == "":
                    continue
                channel = header[j]
                channel_id = self._channel_to_id[channel]

                total_data += 1
                if mask[bin_id][channel_id] == 1:
                    unused_data += 1
                mask[bin_id][channel_id] = 1

                write(data, bin_id, channel, row[j], begin_pos)
                original_value[bin_id][channel_id] = row[j]

        # impute missing values

        if self._impute_strategy not in ['zero', 'normal_value', 'previous', 'next']:
            raise ValueError("impute strategy is invalid")

        if self._impute_strategy in ['normal_value', 'previous']:
            prev_values = [[] for i in range(len(self._id_to_channel))]
            for bin_id in range(N_bins):
                for channel in self._id_to_channel:
                    channel_id = self._channel_to_id[channel]
                    if mask[bin_id][channel_id] == 1:
                        prev_values[channel_id].append(original_value[bin_id][channel_id])
                        continue
                    if self._impute_strategy == 'normal_value':
                        imputed_value = self._normal_values[channel]
                    if self._impute_strategy == 'previous':
                        if len(prev_values[channel_id]) == 0:
                            imputed_value = self._normal_values[channel]
                        else:
                            imputed_value = prev_values[channel_id][-1]
                    write(data, bin_id, channel, imputed_value, begin_pos)

        if self._impute_strategy == 'next':
            prev_values = [[] for i in range(len(self._id_to_channel))]
            for bin_id in range(N_bins-1, -1, -1):
                for channel in self._id_to_channel:
                    channel_id = self._channel_to_id[channel]
                    if mask[bin_id][channel_id] == 1:
                        prev_values[channel_id].append(original_value[bin_id][channel_id])
                        continue
                    if len(prev_values[channel_id]) == 0:
                        imputed_value = self._normal_values[channel]
                    else:
                        imputed_value = prev_values[channel_id][-1]
                    write(data, bin_id, channel, imputed_value, begin_pos)

        empty_bins = np.sum([1 - min(1, np.sum(mask[i, :])) for i in range(N_bins)])
        self._done_count += 1
        self._empty_bins_sum += empty_bins / (N_bins + eps)
        self._unused_data_sum += unused_data / (total_data + eps)

        if self._store_masks:
            data = np.hstack([data, mask.astype(np.float32)])

        # create new header
        new_header = []
        for channel in self._id_to_channel:
            if self._is_categorical_channel[channel]:
                values = self._possible_values[channel]
                for value in values:
                    new_header.append(channel + "->" + value)
            else:
                new_header.append(channel)

        if self._store_masks:
            for i in range(len(self._id_to_channel)):
                channel = self._id_to_channel[i]
                new_header.append("mask->" + channel)

        new_header = ",".join(new_header)

        return (data, new_header)

    def print_statistics(self):
        print("statistics of discretizer:")
        print("\tconverted {} examples".format(self._done_count))
        print("\taverage unused data = {:.2f} percent".format(100.0 * self._unused_data_sum / self._done_count))
        print("\taverage empty  bins = {:.2f} percent".format(100.0 * self._empty_bins_sum / self._done_count))


class Normalizer:
    def __init__(self, fields=None):
        self._means = None
        self._stds = None
        self._fields = None
        if fields is not None:
            self._fields = [col for col in fields]

        self._sum_x = None
        self._sum_sq_x = None
        self._count = 0

    def _feed_data(self, x):
        x = np.array(x)
        self._count += x.shape[0]
        if self._sum_x is None:
            self._sum_x = np.sum(x, axis=0)
            self._sum_sq_x = np.sum(x**2, axis=0)
        else:
            self._sum_x += np.sum(x, axis=0)
            self._sum_sq_x += np.sum(x**2, axis=0)

    def _save_params(self, save_file_path):
        eps = 1e-7
        with open(save_file_path, "wb") as save_file:
            N = self._count
            self._means = 1.0 / N * self._sum_x
            self._stds = np.sqrt(1.0/(N - 1) * (self._sum_sq_x - 2.0 * self._sum_x * self._means + N * self._means**2))
            self._stds[self._stds < eps] = eps
            pickle.dump(obj={'means': self._means,
                             'stds': self._stds},
                        file=save_file,
                        protocol=2)

    def load_params(self, load_file_path):
        with open(load_file_path, "rb") as load_file:
            if platform.python_version()[0] == '2':
                dct = pickle.load(load_file)
            else:
                dct = pickle.load(load_file, encoding='latin1')
            self._means = dct['means']
            self._stds = dct['stds']

    def transform(self, X):
        if self._fields is None:
            fields = range(X.shape[1])
        else:
            fields = self._fields
        ret = 1.0 * X
        for col in fields:
            ret[:, col] = (X[:, col] - self._means[col]) / self._stds[col]
        return ret

In [3]:
# import os
# import numpy as np
# from PIL import Image
# import pandas as pd 

# import torch
# from torch.utils.data import Dataset
# # import 
# import glob
# import torchvision.transforms as transforms
# from torch.utils.data import DataLoader
# import random


# R_CLASSES  = ['Atelectasis', 'Cardiomegaly', 'Consolidation', 'Edema',
#        'Enlarged Cardiomediastinum', 'Fracture', 'Lung Lesion',
#        'Lung Opacity', 'No Finding', 'Pleural Effusion', 'Pleural Other',
#        'Pneumonia', 'Pneumothorax', 'Support Devices']

# CLASSES = [
#        'Acute and unspecified renal failure', 'Acute cerebrovascular disease',
#        'Acute myocardial infarction', 'Cardiac dysrhythmias',
#        'Chronic kidney disease',
#        'Chronic obstructive pulmonary disease and bronchiectasis',
#        'Complications of surgical procedures or medical care',
#        'Conduction disorders', 'Congestive heart failure; nonhypertensive',
#        'Coronary atherosclerosis and other heart disease',
#        'Diabetes mellitus with complications',
#        'Diabetes mellitus without complication',
#        'Disorders of lipid metabolism', 'Essential hypertension',
#        'Fluid and electrolyte disorders', 'Gastrointestinal hemorrhage',
#        'Hypertension with complications and secondary hypertension',
#        'Other liver diseases', 'Other lower respiratory disease',
#        'Other upper respiratory disease',
#        'Pleurisy; pneumothorax; pulmonary collapse',
#        'Pneumonia (except that caused by tuberculosis or sexually transmitted disease)',
#        'Respiratory failure; insufficiency; arrest (adult)',
#        'Septicemia (except in labor)', 'Shock'
#     ]
                    
# class MIMIC_CXR_EHR_RR_DN(Dataset):
#     def __init__(self, args, metadata_with_labels, ehr_ds, cxr_ds, split='train'):
        
#         self.CLASSES = CLASSES
#         if 'radiology' in args.labels_set:
#             self.CLASSES = R_CLASSES
        
#         self.metadata_with_labels = metadata_with_labels
#         self.discharge_notes_paired = self.metadata_with_labels['discharge_text'].values
#         self.radiology_notes_paired = self.metadata_with_labels['radiology_text'].values
#         self.time_diff = self.metadata_with_labels.time_diff
#         self.lower = self.metadata_with_labels.lower
#         self.upper = self.metadata_with_labels.upper
#         self.cxr_files_paired = self.metadata_with_labels.dicom_id.values
#         self.ehr_files_paired = (self.metadata_with_labels['stay'].values)
#         self.cxr_files_all = cxr_ds.filenames_loaded
#         self.ehr_files_all = ehr_ds.names
#         self.ehr_files_unpaired = list(set(self.ehr_files_all) - set(self.ehr_files_paired))
#         self.ehr_ds = ehr_ds
#         self.cxr_ds = cxr_ds
#         self.args = args
#         self.split = split
#         self.data_ratio = self.args.data_ratio 
      
#         self.paired_times= (self.metadata_with_labels['period_length'].values)
#         self.ehr_paired_list = list(zip(self.ehr_files_paired, self.paired_times))

                
#         if split=='test':
#             self.data_ratio =  1.0
#         elif split == 'val':
#             self.data_ratio =  0.0
        

#     def __getitem__(self, index):
#         lower = self.metadata_with_labels.iloc[index].lower
#         upper = self.metadata_with_labels.iloc[index].upper
#         discharge_note = self.discharge_notes_paired[index]
#         radiology_note = self.radiology_notes_paired[index] 
    
#         # Determine the appropriate EHR DataFrame based on the task
#         if self.args.task == 'decompensation' or self.args.task == 'length-of-stay':
#             ehr_df = self.ehr_paired_list
#         else:
#             ehr_df = self.ehr_files_paired
    
#         # Initialize labels and data
#         ehr_data, labels_ehr = None, None
#         cxr_data, labels_cxr = None, None
    
#         # Handle EHR data loading
#         if 'EHR' in self.args.modalities:
#             ehr_data, labels_ehr = self.ehr_ds.__getitem__(ehr_df[index], lower, upper)
#         else:
#             ehr_data, labels_ehr = np.zeros((1, 10)), np.zeros(self.args.num_classes)
    
#         # Handle CXR data loading
#         if 'CXR' in self.args.modalities:
#             cxr_data, labels_cxr = self.cxr_ds[self.cxr_files_paired[index]]
#         else:
#             cxr_data, labels_cxr = np.zeros((1, 10)), np.zeros(self.args.num_classes)
    
#         return ehr_data, cxr_data, discharge_note, radiology_note, labels_ehr, labels_cxr


        
    
#     def __len__(self):
#         if self.args.task == 'decompensation' or self.args.task == 'length-of-stay':
#             ehr_df = self.ehr_paired_list
#         else:
#             ehr_df = self.ehr_files_paired
#         if 'paired' in self.args.data_pairs:
#             return len(self.ehr_files_paired)
#         elif self.args.data_pairs == 'partial_ehr':
#             return len(self.ehr_files_all)
#         elif self.args.data_pairs == 'radiology':
#             return len(self.cxr_files_all)
#         elif self.args.data_pairs == 'partial_ehr_cxr':
#             return len(self.ehr_files_paired) + int(self.data_ratio * len(self.ehr_files_unpaired)) 
        
# def loadmetadata(args, discharge_notes, radiology_reports):
#     data_dir = args.cxr_data_dir
#     cxr_metadata = pd.read_csv(f'{data_dir}/mimic-cxr-2.0.0-metadata.csv')
#     icu_stay_metadata = pd.read_csv(f'{args.ehr_data_dir}/root/all_stays.csv')
#     columns = ['subject_id', 'stay_id', 'intime', 'outtime', 'hadm_id']

#     cxr_merged_icustays = pd.DataFrame()

#     if 'EHR' in args.modalities and 'CXR' in args.modalities:
#         # Merge EHR and CXR data
#         cxr_merged_icustays = cxr_metadata.merge(icu_stay_metadata[columns], how='inner', on='subject_id')
#         cxr_merged_icustays.intime = pd.to_datetime(cxr_merged_icustays.intime)
#         cxr_merged_icustays.outtime = pd.to_datetime(cxr_merged_icustays.outtime)
#     elif 'EHR' in args.modalities:
#         cxr_merged_icustays = icu_stay_metadata[columns]
#         cxr_merged_icustays['StudyDateTime'] = None
#     elif 'CXR' in args.modalities:
#         cxr_merged_icustays = cxr_metadata
#         cxr_merged_icustays['intime'] = None
#         cxr_merged_icustays['outtime'] = None

#     if 'CXR' in args.modalities:
#         cxr_merged_icustays['StudyTime'] = cxr_merged_icustays['StudyTime'].apply(lambda x: f'{int(float(x)):06}')
#         cxr_merged_icustays['StudyDateTime'] = pd.to_datetime(cxr_merged_icustays['StudyDate'].astype(str) + ' ' + cxr_merged_icustays['StudyTime'].astype(str), format="%Y%m%d %H%M%S")

#         if 'EHR' in args.modalities:
#             cxr_merged_icustays.intime = pd.to_datetime(cxr_merged_icustays.intime)
#             cxr_merged_icustays.outtime = pd.to_datetime(cxr_merged_icustays.outtime)
    
#             cxr_merged_icustays['time_diff'] = cxr_merged_icustays.StudyDateTime - cxr_merged_icustays.intime
#             cxr_merged_icustays['time_diff'] = cxr_merged_icustays['time_diff'].apply(lambda x: np.round(x.total_seconds() / 60 / 60, 3))
    
#             cxr_merged_icustays['full_stay_time'] = cxr_merged_icustays.outtime - cxr_merged_icustays.intime
#             cxr_merged_icustays['full_stay_time'] = cxr_merged_icustays['full_stay_time'].apply(lambda x: np.round(x.total_seconds() / 60 / 60, 3))

#     if 'RR' in args.modalities or 'DN' in args.modalities:
#         dsrr_merge_columns = ['subject_id', 'hadm_id']
#         dsrr_columns = ['subject_id', 'hadm_id','charttime', 'text']

#     if 'DN' in args.modalities:
#         cxr_merged_icustays = cxr_merged_icustays.merge(discharge_notes[dsrr_columns], how='left', on=dsrr_merge_columns)
#         cxr_merged_icustays.rename(columns={'text': 'discharge_text'}, inplace=True)
#         cxr_merged_icustays.rename(columns={'charttime': 'discharge_charttime'}, inplace=True)
#         cxr_merged_icustays['discharge_charttime'] = pd.to_datetime(cxr_merged_icustays['discharge_charttime'])
#     else:
#         cxr_merged_icustays['discharge_text'] = None

#     if 'RR' in args.modalities:
#         cxr_merged_icustays = cxr_merged_icustays.merge(radiology_reports[dsrr_columns], how='left', on=dsrr_merge_columns)
#         cxr_merged_icustays.rename(columns={'text': 'radiology_text'}, inplace=True)
#         cxr_merged_icustays.rename(columns={'charttime': 'radiology_charttime'}, inplace=True)
#         cxr_merged_icustays['radiology_charttime'] = pd.to_datetime(cxr_merged_icustays['radiology_charttime'])
#     else:
#         cxr_merged_icustays['radiology_text'] = None
    
#     cxr_merged_icustays_during = cxr_merged_icustays

#     if args.task == 'decompensation' or args.task == 'length-of-stay':
#         train_listfile = pd.read_csv(f'/scratch/se1525/mml-ssl/{args.task}/train_listfile.csv')
#         train_listfile.columns = ['stay', 'period_length', 'stay_id', 'y_true', 'intime', 'endtime']
#         test_listfile = pd.read_csv(f'/scratch/se1525/mml-ssl/{args.task}/test_listfile.csv')
#         test_listfile.columns = ['stay', 'period_length', 'stay_id', 'y_true', 'intime', 'endtime']
#         val_listfile = pd.read_csv(f'/scratch/se1525/mml-ssl/{args.task}/val_listfile.csv')
#         val_listfile.columns = ['stay', 'period_length', 'stay_id', 'y_true', 'intime', 'endtime']
#         listfile = train_listfile.append(test_listfile)
#         listfile = listfile.append(val_listfile)
#         listfile['subject_id'] = listfile['stay'].apply(lambda x: x.split("_")[0])

#         columns2 = ['subject_id', 'endtime']
#         listfile['subject_id'] = listfile['subject_id'].astype('int64')
#         cxr_merged_icustays = cxr_merged_icustays.merge(listfile[columns2], how='inner', on='subject_id')
#         cxr_merged_icustays.endtime = pd.to_datetime(cxr_merged_icustays.endtime)
#         if 'CXR' in args.modalities:    
#             cxr_merged_icustays_during = cxr_merged_icustays.loc[
#                 ((cxr_merged_icustays.StudyDateTime >= cxr_merged_icustays.intime) & (cxr_merged_icustays.StudyDateTime <= cxr_merged_icustays.endtime))]
#         if 'DN' in args.modalities:
#             cxr_merged_icustays_during = cxr_merged_icustays_during.loc[
#                 ((cxr_merged_icustays_during['discharge_charttime'] >= cxr_merged_icustays_during.intime)&(cxr_merged_icustays_during['discharge_charttime'] <= cxr_merged_icustays_during.endtime))]
#         if 'RR' in args.modalities:
#                 cxr_merged_icustays_during = cxr_merged_icustays_during.loc[
#                 ((cxr_merged_icustays_during['radiology_charttime'] >= cxr_merged_icustays_during.intime)&(cxr_merged_icustays_during['radiology_charttime'] <= cxr_merged_icustays_during.endtime))]

#     if args.task == 'in-hospital-mortality':
#         end_time = cxr_merged_icustays.intime + pd.DateOffset(hours=48)
#         if 'CXR' in args.modalities:
#             cxr_merged_icustays_during = cxr_merged_icustays.loc[
#                 ((cxr_merged_icustays.StudyDateTime >= cxr_merged_icustays.intime) & (cxr_merged_icustays.StudyDateTime <= end_time))]
#         if 'DN' in args.modalities:
#             print(cxr_merged_icustays_during.head())
#             cxr_merged_icustays_during = cxr_merged_icustays_during.loc[
#                 ((cxr_merged_icustays_during['discharge_charttime'] >= cxr_merged_icustays_during.intime)&(cxr_merged_icustays_during['discharge_charttime'] <= cxr_merged_icustays_during.intime + pd.DateOffset(hours=48)))]
#         if 'RR' in args.modalities:
#             cxr_merged_icustays_during = cxr_merged_icustays_during.loc[
#                 ((cxr_merged_icustays_during['radiology_charttime'] >= cxr_merged_icustays_during.intime)&(cxr_merged_icustays_during['radiology_charttime'] <= cxr_merged_icustays_during.intime + pd.DateOffset(hours=48)))]

#     if args.task == 'phenotyping' or args.task == 'readmission':
#         end_time = cxr_merged_icustays.outtime
#         if 'CXR' in args.modalities:
#             cxr_merged_icustays_during = cxr_merged_icustays.loc[
#             ((cxr_merged_icustays.StudyDateTime >= cxr_merged_icustays.intime) & (cxr_merged_icustays.StudyDateTime <= end_time))]
#         if 'DN' in args.modalities:
#             print(cxr_merged_icustays_during.head())
#             cxr_merged_icustays_during = cxr_merged_icustays_during.loc[
#                 ((cxr_merged_icustays_during['discharge_charttime'] >= cxr_merged_icustays_during.intime)&(cxr_merged_icustays_during['discharge_charttime'] <= cxr_merged_icustays_during.outtime))]
#         if 'RR' in args.modalities:
#             cxr_merged_icustays_during = cxr_merged_icustays_during.loc[
#                 ((cxr_merged_icustays_during['radiology_charttime'] >= cxr_merged_icustays_during.intime)&(cxr_merged_icustays_during['radiology_charttime'] <= cxr_merged_icustays_during.outtime))]

#     cxr_merged_icustays_AP = cxr_merged_icustays_during[cxr_merged_icustays_during['ViewPosition'] == 'AP']

#     if args.retrieve_cxr == 'recent':
#         groups = cxr_merged_icustays_AP.groupby('stay_id')
#         groups_selected = []
#         for group in groups:
#             # Select the latest CXR for the ICU stay
#             selected = group[1].sort_values('StudyDateTime').tail(1).reset_index()
#             groups_selected.append(selected)
#         groups = pd.concat(groups_selected, ignore_index=True)
#         groups['lower'] = 0
#         groups['upper'] = groups.full_stay_time
#     elif args.retrieve_cxr == 'all':
#         print("All CXR")
#         groups = cxr_merged_icustays_AP.groupby('study_id').first()
#         groups = groups.reset_index()
#         groups = groups.groupby('study_id').first().sort_values(by=['stay_id', 'StudyDateTime'])
#         groups = groups.reset_index()
#         groups['lower'] = 0
#         groups['upper'] = groups.time_diff

#     return groups


# def load_cxr_ehr_rr_dn(args, ehr_train_ds, ehr_val_ds, cxr_train_ds, cxr_val_ds, ehr_test_ds, cxr_test_ds):
#     discharge_notes=pd.read_csv('/scratch/baj321/MIMIC-Note/physionet.org/files/mimic-iv-note/2.2/note/discharge.csv')
#     radiology_reports=pd.read_csv('/scratch/baj321/MIMIC-Note/physionet.org/files/mimic-iv-note/2.2/note/radiology.csv')
#     cxr_merged_icustays = loadmetadata(args, discharge_notes, radiology_reports) 

#     splits_labels_train = pd.read_csv(f'{args.ehr_data_dir}/{args.task}/train_listfile.csv')
#     splits_labels_val = pd.read_csv(f'{args.ehr_data_dir}/{args.task}/val_listfile.csv')
#     splits_labels_test = pd.read_csv(f'{args.ehr_data_dir}/{args.task}/test_listfile.csv')
#     train_meta_with_labels = cxr_merged_icustays.merge(splits_labels_train, how='inner', on='stay_id')
#     val_meta_with_labels = cxr_merged_icustays.merge(splits_labels_val, how='inner', on='stay_id')
#     test_meta_with_labels = cxr_merged_icustays.merge(splits_labels_test, how='inner', on='stay_id')
    
#     train_ds = MIMIC_CXR_EHR_RR_DN(args, train_meta_with_labels, ehr_train_ds, cxr_train_ds)
#     val_ds = MIMIC_CXR_EHR_RR_DN(args, val_meta_with_labels, ehr_val_ds, cxr_val_ds, split='val')
#     test_ds = MIMIC_CXR_EHR_RR_DN(args, test_meta_with_labels, ehr_test_ds, cxr_test_ds, split='test')
    
#     if args.task == 'decompensation' or args.task == 'length-of-stay':
#         print("big one")
#         train_dl = DataLoader(train_ds, args.batch_size, shuffle=True, collate_fn=my_collate, pin_memory=True, num_workers=16, drop_last=True)
#         val_dl = DataLoader(val_ds, args.batch_size, shuffle=False, collate_fn=my_collate, pin_memory=True, num_workers=16, drop_last=False)
#         test_dl = DataLoader(test_ds, args.batch_size, shuffle=False, collate_fn=my_collate, pin_memory=True, num_workers=16, drop_last=False)
#     else:
#         train_dl = DataLoader(train_ds, args.batch_size, shuffle=True, collate_fn=my_collate, pin_memory=True, num_workers=16, drop_last=True)
#         val_dl = DataLoader(val_ds, args.batch_size, shuffle=False, collate_fn=my_collate, pin_memory=True, num_workers=16, drop_last=False)
#         test_dl = DataLoader(test_ds, args.batch_size, shuffle=False, collate_fn=my_collate, pin_memory=True, num_workers=16, drop_last=False)

#     return train_dl, val_dl, test_dl


# def printPrevalence(merged_file, args):
#     if args.labels_set == 'pheno':
#         total_rows = len(merged_file)
#         print(merged_file[CLASSES].sum()/total_rows)
#     else:
#         total_rows = len(merged_file)
#         print(merged_file['y_true'].value_counts())
#     # import pdb; pdb.set_trace()

# def my_collate(batch):
#     x = [item[0] for item in batch]
#     pairs = [False if item[1] is None else True for item in batch]
#     img = torch.stack([torch.zeros(3, 224, 224) if item[1] is None else item[1] for item in batch])
#     x, seq_length = pad_zeros(x)
#     discharge_note = [item[2] for item in batch]
#     radiology_note = [item[3] for item in batch]
#     targets_ehr = np.array([item[4] for item in batch])
#     targets_cxr = torch.stack([torch.zeros(14) if item[5] is None else item[5] for item in batch])
#     return [x, img, discharge_note, radiology_note, targets_ehr, targets_cxr, seq_length, pairs]

# def pad_zeros(arr, min_length=None):
#     dtype = arr[0].dtype
#     seq_length = [x.shape[0] for x in arr]
#     max_len = max(seq_length)
#     ret = [np.concatenate([x, np.zeros((max_len - x.shape[0],) + x.shape[1:], dtype=dtype)], axis=0)
#            for x in arr]
#     if (min_length is not None) and ret[0].shape[0] < min_length:
#         ret = [np.concatenate([x, np.zeros((min_length - x.shape[0],) + x.shape[1:], dtype=dtype)], axis=0)
#                for x in ret]
#     return np.array(ret), seq_length


In [4]:
from __future__ import absolute_import
from __future__ import print_function

import numpy as np
import argparse
import os
import imp
import re
from trainers.fusion_trainer import FusionTrainer
from trainers.mmtm_trainer import MMTMTrainer
from trainers.daft_trainer import DAFTTrainer

# from ehr_utils.preprocessing import Discretizer, Normalizer
from datasets.ehr_dataset import get_datasets
from datasets.cxr_dataset import get_cxr_datasets
from datasets.Quatrimodal_Fusion import load_cxr_ehr_rr_dn
from pathlib import Path
import torch

from arguments import args_parser

parser = args_parser()
args = parser.parse_args([ 
'--vision-backbone', 'resnet34' ,
'--modalities', 'EHR-CXR-RR' ,
'--pretraining', 'RR' ,
'--resize', '384' , 
'--task' , 'in-hospital-mortality' ,
'--epochs' , '20' , 
'--batch_size' , '1' , '--lr' , '0.00001' ,
'--mode' , 'train' ,
'--H_mode' , 'relevancy-based-hierarchical' ,
'--order' , 'RR-CXR-EHR' ,
'--fusion_type' , 'None' ,
'--save_dir' , '/scratch/baj321/mml-ssl/checkpoints/' ,
 '--ehr_data_dir', '/scratch/fs999/shamoutlab/data/mimic-iv-extracted',
'--data_pairs', 'paired_ehr_cxr', 
'--fusion_type' , 'uni_ehr', 
'--num_classes' , '1'])


# add more arguments here ...
# args = parser.parse_args()
print(args)

if args.missing_token is not None:
    from trainers.fusion_tokens_trainer import FusionTokensTrainer as FusionTrainer
    
path = Path(args.save_dir)
path.mkdir(parents=True, exist_ok=True)

seed = 1002
torch.manual_seed(seed)
np.random.seed(seed)

def read_timeseries(args):
    path = f'{args.ehr_data_dir}/{args.task}/train/14991576_episode3_timeseries.csv'
    ret = []
    with open(path, "r") as tsfile:
        header = tsfile.readline().strip().split(',')
        assert header[0] == "Hours"
        for line in tsfile:
            mas = line.strip().split(',')
            ret.append(np.array(mas))
    return np.stack(ret)
    

discretizer = Discretizer(timestep=float(args.timestep),
                          store_masks=True,
                          impute_strategy='previous',
                          start_time='zero')


discretizer_header = discretizer.transform(read_timeseries(args))[1].split(',')
cont_channels = [i for (i, x) in enumerate(discretizer_header) if x.find("->") == -1]

normalizer = Normalizer(fields=cont_channels)  # choose here which columns to standardize
normalizer_state = args.normalizer_state
if normalizer_state is None:
    normalizer_state = 'normalizers/ph_ts{}.input_str:previous.start_time:zero.normalizer'.format(args.timestep)
    normalizer_state = os.path.join(os.path.dirname('/scratch/se1525/mml-ssl/medfuse_baseline/'), normalizer_state)
normalizer.load_params(normalizer_state)

ehr_train_ds, ehr_val_ds, ehr_test_ds = get_datasets(discretizer, normalizer, args)

cxr_train_ds, cxr_val_ds, cxr_test_ds = get_cxr_datasets(args)

#print(" ehr_train_ds" , ehr_train_ds[('16918793_episode1_timeseries.csv', 27.0)])



Namespace(H_mode='relevancy-based-hierarchical', align=0.0, batch_size=1, beta_1=0.9, crop=224, cxr_data_dir='/scratch/fs999/shamoutlab/data/physionet.org/files/mimic-cxr-jpg/2.0.0', daft_activation='linear', data_pairs='paired_ehr_cxr', data_ratio=1.0, depth=1, dim=256, dropout=0.0, ehr_data_dir='/scratch/fs999/shamoutlab/data/mimic-iv-extracted', epochs=20, eval=False, fusion='joint', fusion_type='uni_ehr', imputation='previous', labels_set='pheno', layer_after=4, layers=1, load_state=None, load_state_cxr=None, load_state_ehr=None, lr=1e-05, missing_token=None, mmtm_ratio=4, modalities='EHR-CXR-RR', mode='train', network=None, normalizer_state=None, num_classes=1, order='RR-CXR-EHR', patience=15, pretrained=False, pretraining='RR', rec_dropout=0.0, resize=384, resume=False, retrieve_cxr='recent', run_method='fine_tune', save_dir='/scratch/baj321/mml-ssl/checkpoints/', task='in-hospital-mortality', timestep=1.0, vision_backbone='resnet34', vision_num_classes=14)
/scratch/fs999/shamout

In [5]:
train_dl, val_dl, test_dl = load_cxr_ehr_rr_dn(args, ehr_train_ds, ehr_val_ds, cxr_train_ds, cxr_val_ds, ehr_test_ds, cxr_test_ds)

In [6]:
from trainers.DHF_trainer import DHFTrainer

In [7]:
trainer = DHFTrainer(
        train_dl, 
        val_dl, 
        args,
        test_dl
        )

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: baj321. Use `wandb login --relogin` to force relogin


Some weights of the model checkpoint at allenai/longformer-base-4096 were not used when initializing LongformerModel: ['lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.bias']
- This IS expected if you are initializing LongformerModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LongformerModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_rela

In [ ]:
trainer.train()

running for fusion_type uni_ehr
0
starting val epoch 0
['CXR', 'RR', 'EHR']
ok r loss
['CXR', 'RR', 'EHR']
ok r loss
['CXR', 'RR', 'EHR']
ok r loss
['CXR', 'RR', 'EHR']
ok r loss
['CXR', 'RR', 'EHR']
ok r loss
['CXR', 'RR', 'EHR']
ok r loss
['CXR', 'RR', 'EHR']
ok r loss
['CXR', 'RR', 'EHR']
ok r loss
['CXR', 'RR', 'EHR']
ok r loss
['CXR', 'RR', 'EHR']
ok r loss
['CXR', 'RR', 'EHR']
ok r loss
['CXR', 'RR', 'EHR']
ok r loss
['CXR', 'RR', 'EHR']
ok r loss
['CXR', 'RR', 'EHR']
ok r loss
['CXR', 'RR', 'EHR']
ok r loss
['CXR', 'RR', 'EHR']
ok r loss
['CXR', 'RR', 'EHR']
ok r loss
['CXR', 'RR', 'EHR']
ok r loss
['CXR', 'RR', 'EHR']
ok r loss
['CXR', 'RR', 'EHR']
ok r loss
['CXR', 'RR', 'EHR']
ok r loss
['CXR', 'RR', 'EHR']
ok r loss
['CXR', 'RR', 'EHR']
ok r loss
['CXR', 'RR', 'EHR']
ok r loss
['CXR', 'RR', 'EHR']
ok r loss
['CXR', 'RR', 'EHR']
ok r loss
['CXR', 'RR', 'EHR']
ok r loss
['CXR', 'RR', 'EHR']
ok r loss
['CXR', 'RR', 'EHR']
ok r loss
['CXR', 'RR', 'EHR']
ok r loss
['CXR', 'RR', '

In [ ]:
trainer.eval()

In [9]:
from trainers.unimodal_trainer import UnimodalTrainer

In [10]:
ehr_trainer = UnimodalTrainer(
        train_dl, 
        val_dl, 
        args,
        test_dl
        )

Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [2]:
ehr_trainer.train()

NameError: name 'ehr_trainer' is not defined

In [3]:
ehr_trainer.eval()

NameError: name 'ehr_trainer' is not defined

In [11]:
from trainers.staged_Trainer import StagedFusionTrainer

In [12]:
staged_trainer = StagedFusionTrainer(
        train_dl, 
        val_dl, 
        args,
        test_dl
        )

Some weights of the model checkpoint at allenai/longformer-base-4096 were not used when initializing LongformerModel: ['lm_head.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight']
- This IS expected if you are initializing LongformerModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LongformerModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform

In [ ]:
staged_trainer.train()

running for fusion_type early
0
starting val epoch 0
val [0000 / 0020] validation loss: 	0.82340
checkpoint
starting train epoch 0
 epoch [0000 / 0020] [0009/267] eta: 0 Days 6:13:3         lr: 	1.0000E-05 loss: 	0.72092
 epoch [0000 / 0020] [0109/267] eta: 0 Days 0:56:47        lr: 	1.0000E-05 loss: 	0.45294
 epoch [0000 / 0020] [0209/267] eta: 0 Days 0:41:30        lr: 	1.0000E-05 loss: 	0.44077
1
starting val epoch 1
val [0001 / 0020] validation loss: 	0.43915
checkpoint
starting train epoch 1
 epoch [0001 / 0020] [0009/267] eta: 0 Days 0:44:14        lr: 	1.0000E-05 loss: 	0.41174
 epoch [0001 / 0020] [0109/267] eta: 0 Days 0:38:30        lr: 	1.0000E-05 loss: 	0.42249
 epoch [0001 / 0020] [0209/267] eta: 0 Days 0:34:57        lr: 	1.0000E-05 loss: 	0.41504
2
starting val epoch 2
val [0002 / 0020] validation loss: 	0.42763
checkpoint
starting train epoch 2
 epoch [0002 / 0020] [0009/267] eta: 0 Days 0:36:28        lr: 	1.0000E-05 loss: 	0.46224
 epoch [0002 / 0020] [0109/267] eta: 

In [ ]:
# Iterate through the DataLoader and print the contents of each batch
num = 0
for i, (ehr_data, cxr_data, discharge_note, radiology_note, labels_ehr, labels_cxr,seq_length, pairs) in enumerate(train_dl):
    if i == num:
        print(f"Batch {i+1}:")
        print(f"EHR Data: {ehr_data}")
        print(f"CXR Data: {cxr_data}")
        print(f"Discharge Note: {discharge_note}")
        print(f"Radiology Note: {radiology_note}")
        print(f"Labels EHR: {labels_ehr}")
        print(f"Labels CXR: {labels_cxr}")
        print("\n")

    # Optionally, break after the first batch for brevity
    if i == num:
        break


In [ ]:
for i, batch in enumerate(train_dl):
    print(f"Batch {i+1}:")
    print(f"Batch content: {batch}")

    # Optionally, break after the first batch for brevity
    break